In [1]:
import pandas as pd
import numpy as np

In [2]:
import pickle


In [3]:
import sklearn
sklearn.__version__

'1.6.1'

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-tracking")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1748249050214, experiment_id='1', last_update_time=1748249050214, lifecycle_stage='active', name='nyc-taxi-tracking', tags={}>

In [7]:
df = pd.read_parquet('/workspaces/mlops-zoomcamp/02-experiment-tracking/data/yellow_tripdata_2023-01 (1).parquet')
len(df.columns)

19

In [8]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df["duration"] = df["duration"].apply(lambda x: x.total_seconds()/60)
(df["duration"].iloc[:]).describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [9]:
df_1 = df[(df.duration >= 1) & (df.duration <= 60)]
df_1["duration"].mean()

14.204864143737906

In [10]:
len(df_1)/len(df)

0.9812202822125979

In [11]:
df = df_1

In [12]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3009173 entries, 0 to 3066765
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           object        
 8   DOLocationID           object        
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [14]:
categorical

['PULocationID', 'DOLocationID']

In [15]:
train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train.shape

(3009173, 515)

In [16]:
target = 'duration'
y_train = df[target].values


In [17]:
"""lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)"""

'lr = LinearRegression()\nlr.fit(X_train, y_train)\n\ny_pred = lr.predict(X_train)'

In [18]:
"""np.sqrt(mean_squared_error(y_train, y_pred))"""

'np.sqrt(mean_squared_error(y_train, y_pred))'

In [19]:
"""with open("models/lin_reg.bin", "wb") as f_close:
    pickle.dump((dv,lr), f_close)"""

'with open("models/lin_reg.bin", "wb") as f_close:\n    pickle.dump((dv,lr), f_close)'

In [ ]:
with mlflow.start_run():
    
    mlflow.set_tag("developer", "Priyadharshan")
    mlflow.log_param("train_data_path", "/workspaces/mlops-zoomcamp/02-experiment-tracking/data/yellow_tripdata_2023-01 (1).parquet")
    mlflow.log_param("test_data_path", "/workspaces/mlops-zoomcamp/02-experiment-tracking/data/yellow_tripdata_2023-01 (1).parquet")
    
    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    
    lasso_lr = Lasso(alpha)
    lasso_lr.fit(X_train, y_train)
    y_pred = lasso_lr.predict(X_train)
    error = np.sqrt(mean_squared_error(y_train, y_pred))
    mlflow.log_metric("rmse", error)
    mlflow.log_artifact(local_path="/workspaces/mlops-zoomcamp/02-experiment-tracking/models/lin_reg.bin", artifact_path="models_pickle")
    mlflow.sklearn.log_model(lasso_lr,artifact_path="models_mlflows")

2025/05/26 11:38:39 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp74zk3_m7/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.6.1', 'cloudpickle==2.0.0']. Set logging level to DEBUG to see the full traceback. 
2025/05/26 11:38:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [25]:
import mlflow
logged_model = 'runs:/6bc00b8278d94e199ab35c2cffad94b7/models_mlflows'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflows
  flavor: mlflow.sklearn
  run_id: 6bc00b8278d94e199ab35c2cffad94b7

In [27]:
y_pred = loaded_model.predict(X_train)
error = np.sqrt(mean_squared_error(y_train, y_pred))
error

8.015544916926794

In [21]:
df_test = pd.read_parquet("./data/yellow_tripdata_2023-02.parquet")
df_test.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [22]:
df_test['duration'] = df_test.tpep_dropoff_datetime - df_test.tpep_pickup_datetime
df_test["duration"] = df_test["duration"].apply(lambda x: x.total_seconds()/60)

In [23]:
df_test = df_test[(df_test.duration >= 1) & (df_test.duration <= 60)]


In [24]:
df_test[categorical] = df_test[categorical].astype(str)

In [25]:
test_dicts = df_test[categorical].to_dict(orient='records')
X_test = dv.transform(test_dicts)

In [26]:
target = 'duration'
y_test = df_test[target].values

In [27]:
X_test.shape, y_test.shape

((2855951, 515), (2855951,))

In [28]:
import xgboost as xgb

In [29]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [30]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_test, label=y_test)

In [ ]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

: 

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [10:49:01] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.92334                           
[1]	validation-rmse:8.31136                           
[2]	validation-rmse:7.93312                           
[3]	validation-rmse:7.70311                           
[4]	validation-rmse:7.57214                           
[5]	validation-rmse:7.31507                           
[6]	validation-rmse:7.24632                           
[7]	validation-rmse:7.17143                           
[8]	validation-rmse:7.11637                           
[9]	validation-rmse:7.06850                           
[10]	validation-rmse:6.88403                          
[11]	validation-rmse:6.83225                          
[12]	validation-rmse:6.80762                          
[13]	validation-rmse:6.78190                          
[14]	validation-rmse:6.74346                          
[15]	validation-rmse:6.72139                          
[16]	validation-rmse:6.70577                          
[17]	validation-rmse:6.61313                          
[18]	valid